code
- https://www.kaggle.com/code/carlmcbrideellis/zzzs-random-forest-model-starter

data
- https://www.kaggle.com/datasets/carlmcbrideellis/zzzs-lightweight-training-dataset-target

In [1]:
import numpy as np
import pandas as pd
from itertools import groupby
import gc

In [9]:
# binary class data
#train = pd.read_parquet("/kaggle/input/zzzs-lightweight-training-dataset-target/Zzzs_train.parquet")

# multiclass data
train = pd.read_parquet("../../../Kaggle/data/ChildMindInstitute/Zzzs_train_multi.parquet")

# original data
# train = pd.read_parquet('../../../Kaggle/data/ChildMindInstitute/train_series.parquet')
# test  = pd.read_parquet('../../../Kaggle/data/ChildMindInstitute/test_series.parquet')

In [10]:
print(train.head())
# print(test.head())

      series_id  step                 timestamp     anglez    enmo  awake
0  08db4255286f     0  2018-11-05T10:00:00-0400 -30.845301  0.0447      1
1  08db4255286f     1  2018-11-05T10:00:05-0400 -34.181801  0.0443      1
2  08db4255286f     2  2018-11-05T10:00:10-0400 -33.877102  0.0483      1
3  08db4255286f     3  2018-11-05T10:00:15-0400 -34.282101  0.0680      1
4  08db4255286f     4  2018-11-05T10:00:20-0400 -34.385799  0.0768      1


In [11]:
# value count & unique & Null Count
train.info()
print()
print(train.isnull().sum() / len(train) * 100)
print()

for i in train.columns:
    print('\33[103m', i, '\33[0m')
    print(train[i].value_counts())
    uu = str(train[i].unique())
    print(train[i].dtype)
    print('\33[91m' + uu +  '\033[0m')
    print('\33[94m' + 'Null Count :' + '\033[0m', train[i].isnull().sum())
    print('🌈')
    print()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16289820 entries, 0 to 16289819
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   series_id  object 
 1   step       uint32 
 2   timestamp  object 
 3   anglez     float32
 4   enmo       float32
 5   awake      int64  
dtypes: float32(2), int64(1), object(2), uint32(1)
memory usage: 559.3+ MB

series_id    0.0
step         0.0
timestamp    0.0
anglez       0.0
enmo         0.0
awake        0.0
dtype: float64

 series_id 
series_id
a9a2f7fac455    634680
3452b878e596    535500
18b61dd5aae8    495540
29c75c018220    464220
08db4255286f    440280
89bd631d1769    429300
89c7daa72eee    419580
2fc653ca75c7    416880
a3e59c2ce3f6    416520
939932f1822d    414540
76237b9406d5    412920
55a47ff9dc8a    412740
af91d9a50547    411840
1087d7b0ff2e    410940
e11b9d69f856    408060
7822ee8fe3ec    408060
5acc9d63b5fd    407880
8e32047cbc1f    405540
5f94bb3e1bed    404280
c7b1283bb7eb    404280
b364205aba43    40

Feature Engineering

In [12]:
def make_features(df):
    # parse the timestamp and create an "hour" feature
    df['timestamp'] = pd.to_datetime(df['timestamp']).apply(lambda t: t.tz_localize(None))
    df["hour"] = df["timestamp"].dt.hour
    
    periods = 20
    df["anglez"] = abs(df["anglez"])
    df["anglez_diff"] = df.groupby('series_id')['anglez'].diff(periods=periods).fillna(method="bfill").astype('float16')
    df["enmo_diff"] = df.groupby('series_id')['enmo'].diff(periods=periods).fillna(method="bfill").astype('float16')
    df["anglez_rolling_mean"] = df["anglez"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_mean"] = df["enmo"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_rolling_max"] = df["anglez"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_max"] = df["enmo"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_rolling_std"] = df["anglez"].rolling(periods,center=True).std().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_rolling_std"] = df["enmo"].rolling(periods,center=True).std().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_diff_rolling_mean"] = df["anglez_diff"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_diff_rolling_mean"] = df["enmo_diff"].rolling(periods,center=True).mean().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["anglez_diff_rolling_max"] = df["anglez_diff"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    df["enmo_diff_rolling_max"] = df["enmo_diff"].rolling(periods,center=True).max().fillna(method="bfill").fillna(method="ffill").astype('float16')
    
    return df

features = ["hour",
            "anglez",
            "anglez_rolling_mean",
            "anglez_rolling_max",
            "anglez_rolling_std",
            "anglez_diff",
            "anglez_diff_rolling_mean",
            "anglez_diff_rolling_max",
            "enmo",
            "enmo_rolling_mean",
            "enmo_rolling_max",
            "enmo_rolling_std",
            "enmo_diff",
            "enmo_diff_rolling_mean",
            "enmo_diff_rolling_max",
           ]

Training

In [13]:
train = make_features(train)

X_train = train[features]
y_train = train["awake"]

# save some memory
del train
gc.collect();

In [14]:
%%time

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=50,
                                    min_samples_leaf=300,
                                    random_state=42,
                                    n_jobs=-1)

classifier.fit(X_train, y_train)

# save some memory
del X_train, y_train
gc.collect();

CPU times: total: 1h 27min 21s
Wall time: 6min 1s


In [15]:
test  = pd.read_parquet("../../../Kaggle/data/ChildMindInstitute/test_series.parquet")

test = make_features(test)

X_test = test[features]

test["not_awake"] = classifier.predict_proba(X_test)[:,0]
test["awake"]     = classifier.predict_proba(X_test)[:,1]

preprocessing

In [16]:
# smoothing the predictions
smoothing_length = 2*230
test["score"]  = test["awake"].rolling(smoothing_length,center=True).mean().fillna(method="bfill").fillna(method="ffill")
test["smooth"] = test["not_awake"].rolling(smoothing_length,center=True).mean().fillna(method="bfill").fillna(method="ffill")
# re-binarize
test["smooth"] = test["smooth"].round()

# https://stackoverflow.com/questions/73777727/how-to-mark-start-end-of-a-series-of-non-null-and-non-0-values-in-a-column-of-a
def get_event(df):
    lstCV = zip(df.series_id, df.smooth)
    lstPOI = []
    for (c, v), g in groupby(lstCV, lambda cv: 
                            (cv[0], cv[1]!=0 and not pd.isnull(cv[1]))):
        llg = sum(1 for item in g)
        if v is False: 
            lstPOI.extend([0]*llg)
        else: 
            lstPOI.extend(['onset']+(llg-2)*[0]+['wakeup'] if llg > 1 else [0])
    return lstPOI

test["event"] = get_event(test)

submission

In [17]:
sample_submission = test.loc[test["event"] != 0][["series_id","step","event","score"]].copy().reset_index(drop=True).reset_index(names="row_id")


In [18]:
sample_submission.to_csv('submission_1.csv', index=False)
